In [35]:
import pandas as pd
import re
import string
from datetime import datetime
import joblib
import numpy as np



PATH = "D:/Repository/PetaKabar/whowherewhen/"
bencana = "bencana/"
ekonomi = "ekonomi/"
kecelakaan = "kecelakaan/"
kesehatan = "kesehatan/"
kriminalitas = "kriminalitas/"
olahraga = "olahraga/"

model_location = PATH

news = pd.read_csv('perbandingan/berita-24-5.csv')

In [36]:
# news[news['ner_neg'].isna() == False]['ner_neg'].value_counts()

In [37]:
iter_through = [
    'bencana',
    'ekonomi', 'kecelakaan', 'kesehatan',
    'kriminalitas',
    'olahraga'
    ]

prov = joblib.load(PATH + 'provinsi.pkl')
kab = joblib.load(PATH + 'kabupaten.pkl')
kec = joblib.load(PATH + 'kecamatan.pkl')
daerah = joblib.load(PATH + 'listProvKabKec.pkl')
negara = joblib.load(PATH + 'list_negara.pkl')

day = ("senin", 'selasa', 'rabu', 'kamis', 'jumat', 'sabtu', 'minggu')
month = ('januari', 'februari', 'maret', 'april', 'mei', 'juni', 'juli', 'agustus', 'september', 'oktober', 'november', 'desember')
kata_excl = ('usd', 'rp', 'timur', 'pusat', 'barat', 'utara', 'selatan', 'tengah', 'tenggara', 'kecamatan', 'kelurahan', 'desa', 'kabupaten', 'kota', 'dusun', 'rt', 'kampung','provinsi', 'jawa', 'nusa', 'kalimantan', 'sulawesi', 'papua' ,'aceh', 'jakarta', 'lombok', 'sumba', 'barito', 'surakarta', 'jalan')
jan = ('januari', 'jan', 'january')
feb = ('februari', 'feb', 'pebruari', 'peb', 'february')
mar = ('maret', 'mar', 'march')
apr = ('april', 'apr')
mei = ('mei', 'may')
jun = ('juni', 'jun', 'june')
jul = ('juli', 'jul', 'july')
agu = ('agustus', 'august', 'agu', 'aug')
sep = ('september', 'sept', 'sep')
okt = ('oktober', 'okt', 'oct', 'october')
nov = ('november', 'nov', 'nopember', 'nop')
des = ('desember', 'des', 'dec', 'december')

pattern_date = [('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), #Kamis (1/12/2022)
                ('PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jakarta, Minggu, 4 Desember 2022,
                ('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), # Dirangkum Minggu (25/12),
                ('PROPN', 'NOUN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jumat malam, 2 Desember 2022.
                ('PROPN', 'PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Selasa pagi, 6 Desember 2022.
                ('NUM', 'PROPN', 'NUM'), # 7 Desember 2022
                ('NUM', 'PROPN', 'PUNCT'), # 22 Desember, 
                ('NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM') # 2022/12/14 OR 14/12/2022
                ]
idx_date = [(0, 2, 4, 6),
            (0, 2, 3, 4),
            (0, 2, 4),
            (0, 3, 4, 5),
            (0, 3, 4, 5),
            (-1, 0, 1, 2),
            (-1, 0, 1, -1),
            (-1, 0, 2, 4)
            ]

In [38]:
# daerah dengan kecamatan dan provinsi yang sama tetapi memiliki kabupaten lebih dari 1 dan berbeda
for x in daerah:
    flag = False
    for y in daerah:
        if x[0] == y[0] and x[2] == y[2]:
            if flag:
                print("kec prov ---",x[0], x[1], x[2])
            else:
                flag = True

kec prov --- banten cilegon cibeber
kec prov --- banten lebak cibeber
kec prov --- banten lebak sobang
kec prov --- banten pandeglang sobang
kec prov --- banten serang curug
kec prov --- banten tangerang curug
kec prov --- di yogyakarta bantul jetis
kec prov --- di yogyakarta yogyakarta jetis
kec prov --- jawa barat bandung barat cipeundeuy
kec prov --- jawa barat bandung barat cisarua
kec prov --- jawa barat bandung barat cisarua
kec prov --- jawa barat bandung pacet
kec prov --- jawa barat bandung paseh
kec prov --- jawa barat bandung majalaya
kec prov --- jawa barat bandung banjaran
kec prov --- jawa barat bandung pameungpeuk
kec prov --- jawa barat bandung lengkong
kec prov --- jawa barat bandung sukasari
kec prov --- jawa barat bandung sukasari
kec prov --- jawa barat bandung sukasari
kec prov --- jawa barat bandung cidadap
kec prov --- jawa barat bekasi sukatani
kec prov --- jawa barat bekasi cibitung
kec prov --- jawa barat bogor cibinong
kec prov --- jawa barat bogor tanjungsar

In [39]:
# daerah dengan kecamatan dan kabupaten yang sama tetapi memiliki provinsi lebih dari 1 dan berbeda
for x in daerah:
    flag = False
    for y in daerah:
        if x[2] == y[2] and x[1] == y[1]:
            if flag:
                print("kec kab ---",x[0], x[1], x[2])
            else:
                flag = True

In [40]:
# daerah dengan kecamatan dan kabupaten yang identik dan memiliki daerah dengan kabupaten yang sama namun provinsi berbeda
for x in daerah:
    if x[1] != x[2]:
        continue
    # flag = False
    # base = ""
    for y in daerah:
        if x[1] == y[1] and x[0] != y[0]:
        # if flag:
        #     print("base ------", base)
            print("kec kab -----",x[0],x[1], x[2])
            print("kec kab 2 ---",y[0],y[1], y[2])
        # else:
        #     flag = True
        #     base = str(x[0] + " " + x[1] + " " + x[2])

kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar martapura kota
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar sei/sungai pinang
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar peramasan
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar sambung makmur
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar pengaron
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar telaga bauntung
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar simpang empat
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar mataraman
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar aranio
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar astambul
kec kab ----- jawa barat banjar banjar
kec kab 2 --- kalimantan selatan banjar karang i

In [41]:
# daerah dengan kecamatan dan kabupaten yang identik dan memiliki daerah dengan kecamatan yang sama namun provinsi berbeda
for x in daerah:
    if x[1] != x[2]:
        continue
    # flag = False
    # base = ""
    for y in daerah:
        if x[2] == y[2] and x[0] != y[0]:
        # if flag:
        #     print("base ------", base)
            print("kec kab -----",x[0],x[1], x[2])
            print("kec kab 2 ---",y[0],y[1], y[2])
        # else:
        #     flag = True
        #     base = str(x[0] + " " + x[1] + " " + x[2])

kec kab ----- jawa barat banjar banjar
kec kab 2 --- bali buleleng banjar
kec kab ----- jawa barat banjar banjar
kec kab 2 --- banten pandeglang banjar
kec kab ----- jawa barat purwakarta purwakarta
kec kab 2 --- banten cilegon purwakarta
kec kab ----- jawa barat sukabumi sukabumi
kec kab 2 --- lampung bandar lampung sukabumi
kec kab ----- jawa tengah banyumas banyumas
kec kab 2 --- lampung pringsewu banyumas
kec kab ----- jawa tengah batang batang
kec kab 2 --- sulawesi selatan jeneponto batang
kec kab ----- jawa tengah karanganyar karanganyar
kec kab 2 --- jawa timur ngawi karanganyar
kec kab ----- jawa tengah kendal kendal
kec kab 2 --- jawa timur ngawi kendal
kec kab ----- jawa tengah purworejo purworejo
kec kab 2 --- jawa timur pasuruan purworejo
kec kab ----- jawa tengah rembang rembang
kec kab 2 --- jawa timur pasuruan rembang
kec kab ----- jawa tengah sukoharjo sukoharjo
kec kab 2 --- lampung pringsewu sukoharjo
kec kab ----- jawa tengah wonosobo wonosobo
kec kab 2 --- lampung 

In [42]:
prov_kab = dict()
for x in daerah:
    prov_kab[x[0]] = set()
for x in daerah:
    prov_kab[x[0]].add(x[1])
print(prov_kab)
for a in prov_kab:
    b = prov_kab[a]
    for c in prov_kab:
        d = prov_kab[c]
        if a == c:
            continue
        for x in d:
            if x in b:
                print(a, c, x)
print("-"*30)
for a in prov_kab:
    b = prov_kab[a]
    for c in prov_kab:
        # if a == c:
        #     continue
        d = prov_kab[c]
        for x in d:
            if c != x:
                continue
            if x in b:
                print(a, c, x)

{'bali': {'buleleng', 'klungkung', 'bangli', 'badung', 'gianyar', 'denpasar', 'jembrana', 'karangasem', 'tabanan'}, 'bangka belitung': {'pangkal pinang', 'belitung timur', 'bangka barat', 'belitung', 'bangka tengah', 'bangka selatan', 'bangka'}, 'banten': {'lebak', 'pandeglang', 'tangerang selatan', 'tangerang', 'cilegon', 'serang'}, 'bengkulu': {'bengkulu selatan', 'kaur', 'bengkulu utara', 'lebong', 'bengkulu', 'rejang lebong', 'muko muko', 'kepahiang', 'bengkulu tengah', 'seluma'}, 'di yogyakarta': {'yogyakarta', 'kulon progo', 'bantul', 'gunung kidul', 'sleman'}, 'dki jakarta': {'jakarta timur', 'kepulauan seribu', 'jakarta pusat', 'jakarta selatan', 'jakarta utara', 'jakarta barat'}, 'gorontalo': {'pohuwato', 'bone bolango', 'gorontalo', 'gorontalo utara', 'boalemo'}, 'jambi': {'bungo', 'tanjung jabung barat', 'tebo', 'batang hari', 'sarolangun', 'merangin', 'muaro jambi', 'jambi', 'tanjung jabung timur', 'kerinci', 'sungaipenuh'}, 'jawa barat': {'indramayu', 'subang', 'garut', 's

In [43]:
def preprocessing(berita):
        s = str(berita)
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ')
        s = s.replace('  ', ' ')
        return s

In [44]:
prov = joblib.load(PATH + 'provinsi.pkl')
kab = joblib.load(PATH + 'kabupaten.pkl')
kec = joblib.load(PATH + 'kecamatan.pkl')
daerah = joblib.load(PATH + 'listProvKabKec.pkl')
negara = joblib.load(PATH + 'list_negara.pkl')

daerah
len(daerah)

6994

In [45]:
day = ("senin", 'selasa', 'rabu', 'kamis', 'jumat', 'sabtu', 'minggu')
month = ('januari', 'februari', 'maret', 'april', 'mei', 'juni', 'juli', 'agustus', 'september', 'oktober', 'november', 'desember')
kata_excl = ('usd', 'rp', 'timur', 'pusat', 'barat', 'utara', 'selatan', 'tengah', 'tenggara', 'kecamatan', 'kelurahan', 'desa', 'kabupaten', 'kota', 'dusun', 'rt', 'kampung','provinsi', 'jawa', 'nusa', 'kalimantan', 'sulawesi', 'papua' ,'aceh', 'jakarta', 'lombok', 'sumba', 'barito', 'surakarta', 'jalan')
jan = ('januari', 'jan', 'january')
feb = ('februari', 'feb', 'pebruari', 'peb', 'february')
mar = ('maret', 'mar', 'march')
apr = ('april', 'apr')
mei = ('mei', 'may')
jun = ('juni', 'jun', 'june')
jul = ('juli', 'jul', 'july')
agu = ('agustus', 'august', 'agu', 'aug')
sep = ('september', 'sept', 'sep')
okt = ('oktober', 'okt', 'oct', 'october')
nov = ('november', 'nov', 'nopember', 'nop')
des = ('desember', 'des', 'dec', 'december')

pattern_date = [('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), #Kamis (1/12/2022)
                ('PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jakarta, Minggu, 4 Desember 2022,
                ('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), # Dirangkum Minggu (25/12),
                ('PROPN', 'NOUN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jumat malam, 2 Desember 2022.
                ('PROPN', 'PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Selasa pagi, 6 Desember 2022.
                ('NUM', 'PROPN', 'NUM'), # 7 Desember 2022
                ('NUM', 'PROPN', 'PUNCT'), # 22 Desember, 
                ('NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM') # 2022/12/14 OR 14/12/2022
                ]
idx_date = [(0, 2, 4, 6),
            (0, 2, 3, 4),
            (0, 2, 4),
            (0, 3, 4, 5),
            (0, 3, 4, 5),
            (-1, 0, 1, 2),
            (-1, 0, 1, -1),
            (-1, 0, 2, 4)
            ]

def clean_num(text):
    return re.sub('\D', '', text)

def represents_int(s):
    try: 
        int(s)
    except ValueError:
        return False
    else:
        return True

def handle_bulan(text):
    if represents_int(text):
        if int(text) < 13 and int(text) > 0:
            return int(text)
    else:
        text = text.lower()
        if text in jan:
            return 1
        if text in feb:
            return 2
        if text in mar:
            return 3
        if text in apr:
            return 4
        if text in mei:
            return 5
        if text in jun:
            return 6
        if text in jul:
            return 7
        if text in agu:
            return 8
        if text in sep:
            return 9
        if text in okt:
            return 10
        if text in nov:
            return 11
        if text in des:
            return 12
    return -1

def handle_tahun(text):
    # print(text)
    # print(type(text))
    text = clean_num(text)
    if represents_int(text):
        if int(text) < 2050 and int(text) > 2000:
            return int(text)
    return -1

def handle_tanggal(text):
    # print(text)
    # print(type(text))
    if len(text.split("-")) == 2:
        text = text.split("-")[0]
    text = clean_num(text)
    if represents_int(text):
        if int(text) < 32 and int(text) > 0:
            return int(text)
    print(text, type(text))
    return 0

def write_date(tgl, bulan, tahun):
    if tgl/10 < 1:
        tgl = "0" + str(tgl)     
    if bulan/10 < 1:
        bulan = "0" + str(bulan)
    return str(tgl)+"/"+str(bulan)+"/"+str(tahun)  

def find_date(sentence=None):
    hari = []
    tgl = []
    bulan = []
    tahun = []
    len_sentence = len(sentence)
    for j in range(len_sentence):

        for i in range(len(pattern_date)):
            len_pattern = len(pattern_date[i])
            if len_sentence - j < len_pattern: # untuk jaga-jaga out of index
                break
            flag = True # kalau true ada yg masuk dari 1 pattern
            
            for k in range(len_pattern):
                if sentence.tokens[j+k].tag != pattern_date[i][k]:
                    flag = False
                    break
            
            if flag:
                if idx_date[i][0] == -1:
                    if handle_bulan(sentence.tokens[j + idx_date[i][2]].text) != -1:
                        if idx_date[i][-1] == -1 and handle_tanggal(sentence.tokens[j + idx_date[i][1]].text) > 0 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) > 0:
                            hari.append('-')
                            tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                            bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                            tahun.append(-999)
                        elif handle_tanggal(sentence.tokens[j + idx_date[i][1]].text) > 0 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) > 0 and handle_tahun(sentence.tokens[j + idx_date[i][3]].text) > 0:
                            hari.append('-')
                            tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                            bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                            tahun.append(handle_tahun(sentence.tokens[j + idx_date[i][3]].text))
                        elif handle_tanggal(sentence.tokens[j + idx_date[i][3]].text) > 0 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) > 0 and handle_tahun(sentence.tokens[j + idx_date[i][1]].text) > 0:
                            hari.append('-')
                            tahun.append(handle_tahun(sentence.tokens[j + idx_date[i][1]].text))
                            bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                            tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][3]].text))
                    else:
                        continue
                elif str(sentence.tokens[j + idx_date[i][0]].text).lower() not in day:
                    continue
                elif len(idx_date[i]) == 4 and handle_tanggal(sentence.tokens[j + idx_date[i][1]].text) > 0 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) > 0 and handle_tahun(sentence.tokens[j + idx_date[i][3]].text) > 0:
                    hari.append(str(sentence.tokens[j + idx_date[i][0]].text))
                    tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                    bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                    tahun.append(handle_tahun(sentence.tokens[j + idx_date[i][3]].text))
                elif len(idx_date[i]) == 3 and handle_tanggal(sentence.tokens[j + idx_date[i][1]].text) > 0 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) > 0:
                    hari.append(str(sentence.tokens[j + idx_date[i][0]].text))
                    tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                    bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                    tahun.append(-999)
        
    return hari, tgl, bulan, tahun

def toOriginal(teks):
        teks = teks.lower()
        hasil=''
        if(teks=='sulsel'):
            hasil='sulawesi selatan'
        elif(teks=='sultra'):
            hasil='sulawesi tenggara'
        elif(teks=='sulut'):
            hasil='sulawesi utara'
        elif(teks=='sulteng'):
            hasil='sulawesi tengah'
        elif(teks=='sulbar'):
            hasil='sulawesi barat'
        elif(teks=='kaltim'):
            hasil='kalimantan timur'
        elif(teks=='kaltara'):
            hasil='kalimantan utara'
        elif(teks=='kalteng'):
            hasil='kalimantan tengah'
        elif(teks=='kalsel'):
            hasil='kalimantan selatan'
        elif(teks=='kalbar'):
            hasil='kalimantan barat'
        elif(teks=='jatim'):
            hasil='jawa timur'
        elif(teks=='jateng'):
            hasil='jawa tengah'
        elif(teks=='solo' or teks=='surakarta' ):
            hasil='surakarta (solo)'
        elif(teks=='jabar'):
            hasil='jawa barat'
        elif(teks=='diy' or teks =='yogyakarta'):
            hasil='di yogyakarta'
        elif(teks=='jakbar'):
            hasil='jakarta barat'
        elif(teks=='jaktim'):
            hasil='jakarta timur'
        elif(teks=='jaksel'):
            hasil='jakarta selatan'
        elif(teks=='jakpus'):
            hasil='jakarta pusat'
        elif(teks=='jakut'):
            hasil='jakarta utara'
        elif(teks=='nad'):
            hasil='nanggroe aceh darussalam'
        elif(teks=='sumsel'):
            hasil='sumatera selatan'
        elif(teks=='sumut'):
            hasil='sumatera utara'
        elif(teks=='sumbar'):
            hasil='sumatera barat'
        elif(teks=='jakarta'):
            hasil='dki jakarta'
        else:
            hasil=teks
        return hasil

def clean_koma(list, idx_list):
    val = []
    idx_val = []
    for j in range(len(list)):
        splitted = list[j].split(",")
        if len(splitted) == 1:
            val.append(toOriginal(list[j]))
            idx_val.append(idx_list[j])
        else:
            for i in range(len(splitted)):
                val.append(toOriginal(splitted[i]))
                idx_val.append(idx_list[j])
    return val, idx_val


def ngrams_maker(words, idx_list):
    val = []
    left = []
    right = []
    i = 0
    temp = ""
    while i < len(words):
        if i+1 < len(words) and idx_list[i+1] - 1 == idx_list[i]:
            if temp:
                temp1 = " ".join([temp, words[i+1]])
                temp = temp1
            else:
                temp = " ".join([words[i], words[i+1]])
                left.append(idx_list[i])
        elif temp:
            val.append(temp)
            temp = ""
            right.append(idx_list[i])
        else:
            val.append(words[i])
            left.append(idx_list[i])
            right.append(idx_list[i])
        i += 1
    return val, left, right

def punct_check(s):
    for ch in s:
        if ch in string.punctuation:
            return False
    return True

def dedup_list(l):
    q = set(l)
    return list(q)

def find_left_tag_text(tagged, idx):
    i = idx - 1
    while i >= 0:
        if tagged.tokens[i].tag == 'PROPN' or tagged.tokens[i].tag == 'CCONJ' or tagged.tokens[i].tag == 'NUM':
            i -= 1 
        else:
            return [tagged.tokens[i].tag, tagged.tokens[i].text, i]
    return ['', '', -1]

def find_right_tag_text(tagged, idx):
    i = idx + 1
    while i < len(tagged):
        if tagged.tokens[i].tag == 'PROPN' or tagged.tokens[i].tag == 'CCONJ' or tagged.tokens[i].tag == 'NUM':
            i += 1 
        else:
            return [tagged.tokens[i].tag, tagged.tokens[i].text, i]
    return ['', '', -1]

# def find_like_location(text):
#     val = []
#     for x in kec:
#         splitted = x.split(" ")
#         if len(splitted) == 3:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#             elif splitted[2] == text:
#                 val.append(x)

#         elif len(splitted) == 2:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#         elif len(splitted) == 1:
#             if splitted[0] == text:
#                 val.append(x)
#     for x in kab:
#         splitted = x.split(" ")
#         if len(splitted) == 3:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#             elif splitted[2] == text:
#                 val.append(x)

#         elif len(splitted) == 2:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#         elif len(splitted) == 1:
#             if splitted[0] == text:
#                 val.append(x)
#     for x in prov:
#         splitted = x.split(" ")
#         if len(splitted) == 3:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#             elif splitted[2] == text:
#                 val.append(x)

#         elif len(splitted) == 2:
#             if splitted[0] == text:
#                 val.append(x)
#             elif splitted[1] == text:
#                 val.append(x)
#         elif len(splitted) == 1:
#             if splitted[0] == text:
#                 val.append(x)
#     return val

# res = [[0, 'a'], [0,'b'], [1,'c'], [3, 'd'], [0, 'e']]
# temp = []
# flag_idx = []
# remove_idx = []
# for j in range(len(res)):
#     if res[j][0] not in flag_idx:
#         flag_idx.append(res[j][0])
#     else:
#         remove_idx.append(j)
# for j in range(len(res)):
#     if j in remove_idx:
#         continue
#     temp.append(res[j])
# res = temp
# print(res)

# print(ngrams_maker(['q', 'w', 'e', 'r', 'y'], [0, 1, 2, 4, 5]))
# print(punct_check('a-a'))
# print(dedup_list(['q', 'w', 'e', 'r', 'y', 'y']))

# tm = '2018-11-12 09:15:32'
# tahun = datetime.strptime(tm, f'%Y-%m-%d %H:%M:%S').year
# bulan = datetime.strptime(tm, f'%Y-%m-%d %H:%M:%S').month
# tgl = datetime.strptime(tm, f'%Y-%m-%d %H:%M:%S').day
# result = write_date(tgl, bulan, tahun)
# print(result)

# print(find_date())


In [48]:
iter_through = [
    'bencana',
    'ekonomi', 'kecelakaan', 'kesehatan',
    'kriminalitas',
    'olahraga'
    ]
# tagged = joblib.load(PATH + 'test_tagged_' + n + '.pkl')
# berita = news[news['berita_topik_id'] == 1]

for topik_id, n in enumerate(iter_through):
    tagged = joblib.load(PATH + 'test_tagged_' + n + '.pkl')
    berita = joblib.load('D:/Repository/PetaKabar/perbandingan/' + n + '_test_unpreprocessed.pkl')

    # ----------------------------------- WHEN

    dates = []
    sentences = []

    # find date
    for i in range(len(tagged)):
        result = '-'

        # print(tagged[i])
        sentences.append(tagged[i])
        sentence = tagged[i]
        hari, tgl, bulan, tahun = find_date(sentence)
        if hari:
            idx = -1
            max = 0
            tahun_now = datetime.strptime(berita['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').year
            for j in range(len(tahun)):
                if tahun[j] == -999:
                    tahun[j] = tahun_now
                if max < tahun[j]:
                    max = tahun[j]
            for j in range(len(tahun)):
                if tahun[j] != -1 and max == tahun[j]:
                    tahun = tahun[j]
                    idx = j
                    break
            if idx == -1 or tahun == -999:
                tahun = datetime.strptime(berita['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').year
                tgl = tgl[0]
                bulan = bulan[0]
            else:
                tgl = tgl[idx]
                bulan = bulan[idx]
            result = write_date(tgl, bulan, tahun)
            # print(result)
        # else:
        #     print('-----')
        if result == '-':
    # for j in range(0, len(tagged[i])):
    #     if tagged[i].tokens[j].text.lower() == 'kemarin' or tagged[i].tokens[j].text.lower() == 'lalu':
            # print(tagged[i])
            # break
            tahun = datetime.strptime(berita['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').year
            bulan = datetime.strptime(berita['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').month
            tgl = datetime.strptime(berita['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').day
            result = write_date(tgl, bulan, tahun)
        dates.append(result)

    # df_dates.to_csv(PATH + 'date_tagged_result_v2_' + n + '.csv')

    # ----------------------------------- WHERE
    locations = []
    countries = [] # untuk ekonomi

    for i in range(len(tagged)):
        # print(tagged[i])
        propn_list = [] # untuk semua 'propn'
        idx_list = []
        adp_idx = [] # untuk 'di' + propn
        
        # save each propn words to list
        for j in range(0, len(tagged[i])):
            if tagged[i].tokens[j].tag == 'PROPN':
                idx_list.append(j)
                propn_list.append(tagged[i].tokens[j].text)
            if tagged[i].tokens[j].text == 'di' or tagged[i].tokens[j].text == 'ke':
                adp_idx.append(j)
        # print(adp_idx)
        # create bigrams and trigrams
        bigrams = []
        bigrams_idx = []
        trigrams = []
        trigrams_idx = []

        # check bigrams
        for j in range(1,len(idx_list)):
            if idx_list[j] - 1 == idx_list[j-1]:
                bigrams.append([propn_list[j-1], propn_list[j]])
                bigrams_idx.append([idx_list[j-1], idx_list[j]])

        for j in range(1, len(bigrams_idx)):
            if bigrams_idx[j][0] == bigrams_idx[j-1][1]:
                trigrams.append([bigrams[j-1][0], bigrams[j-1][1], bigrams[j][1]])
                trigrams_idx.append([bigrams_idx[j-1][0], bigrams_idx[j-1][1], bigrams_idx[j][1]])

        def check_list(text, list):
            val = text.lower()
            for x in list:
                if val == x:
                    return True
            return False
        
        propn_list, idx_list = clean_koma(propn_list, idx_list)
        res = []

        # check kec
        for j in range(len(trigrams)):
            joined = " ".join(trigrams[j])
            if check_list(toOriginal(joined), kec) or check_list(toOriginal(joined), kab) or check_list(toOriginal(joined), prov):
                res.append([trigrams_idx[j][0], toOriginal(joined)])
        for j in range(len(bigrams)):
            joined = " ".join(bigrams[j])
            if check_list(toOriginal(joined), kec) or check_list(toOriginal(joined), kab) or check_list(toOriginal(joined), prov):
                res.append([bigrams_idx[j][0], toOriginal(joined)])
        for j in range(len(propn_list)):
            if check_list(propn_list[j], kec) or check_list(propn_list[j], kab) or check_list(propn_list[j], prov):
                res.append([idx_list[j], propn_list[j]])
        
        # remove child grams if there's bigram or trigram
        temp = []
        flag_idx = []
        remove_idx = []
        for j in range(len(res)):
            if res[j][0] not in flag_idx:
                flag_idx.append(res[j][0])
            else:
                remove_idx.append(j)
        for j in range(len(res)):
            if j in remove_idx:
                continue
            temp.append(res[j])
        res = temp

        # print(res)
        res_lower = []
        for j in range(len(res)):
            res_lower.append(res[j][1].lower())

        kec_idx = []
        kab_idx = []
        prov_idx = []
        # catch frasa Kecamatan, Kabupaten, Provinsi
        for j in range(len(propn_list)-2):
            if propn_list[j] == 'kecamatan':
                kec_idx.append(idx_list[j])
            elif propn_list[j] == 'kabupaten' or propn_list[j] == 'kota':
                kab_idx.append(idx_list[j])
            elif propn_list[j] == 'provinsi':
                prov_idx.append(idx_list[j])
        # eval/validate found lists
        trueKec = []
        trueKab = []
        trueProv = []
        
        for loc in res:
            if loc[0] - 1 in kec_idx:
                trueKec.append(loc[1])
            elif loc[0] - 1 in kab_idx:
                trueKab.append(loc[1])
            elif loc[0] - 1 in prov_idx:
                trueProv.append(loc[1])
        # print([(trueKec), (trueKab), (trueProv)])
        trueLocation = []
        for x in daerah:
            for loc in trueKec:
                if loc.lower() == x[2]:
                    if x[1] in res_lower:
                        trueLocation.append([x[2], x[1], x[0]])
            for loc in trueKab:
                if loc.lower() == x[1]:
                    if x[0] in res_lower:
                        trueLocation.append([x[1], x[0]])
                    else:
                        trueLocation.append([x[1], x[0]])
        for x in trueProv:
            trueLocation.append([x.lower()])

        if len(trueLocation) == 0: # jika tidak ada istilah 'Kecamatan', 'Kabupaten' , 'Kota', 'Provinsi'
            for x in daerah: # cek apakah terdapat kabupaten atau provinsi, kecamatan akan masuk juga jika ada kab/prov
                for loc in res_lower:
                    if loc == x[2]: # loc in kecamatan
                        if x[0] in res_lower: # kecamatan provinsi masuk
                            trueLocation.append([x[2], x[1], x[0]])
                        if x[2] == x[1]: # jika kecamatan == kabupaten (ex: sampang sampang)
                            continue
                        if x[1] in res_lower: # kecamatan kabupaten masuk
                            trueLocation.append([x[2], x[1], x[0]])
                    elif loc == x[1]: # loc in kabupaten
                        if x[0] == x[1]: # jika provinsi == kabupaten (ex: jambi jambi)
                            continue
                        if x[0] in res_lower: # kabupaten provinsi masuk
                            trueLocation.append([x[1], x[0]])
                    #     else:
                    #         trueLocation.append([x[1], x[0]])
                    # elif loc == x[0]:
                    #     trueLocation.append([x[0]])
            if len(trueLocation) == 0:
                # print(adp_idx)
                for j in range(len(adp_idx)): # jika terdapat kata 'di' preposisi, maka cek kata setelahnya yg mengandung daerah
                    for k in res:
                        if adp_idx[j] < k[0] and k[0] - adp_idx[j] < 14: #jika jarak lebih dari 13, maka sudah tidak berkaitan
                            for x in daerah:
                                if k[1] == x[0]: #jika provinsi
                                    trueLocation.append([x[0]])
                                elif k[1] == x[1]: #jika kabupaten
                                    trueLocation.append([x[1], x[0]])
                                # elif k[1] == x[2]: #jika kecamatan
                                #     trueLocation.append([x[2], x[1], x[0]])
                # trueLocation.sort(key = len)
            else:
                trueLocation.sort(key = len, reverse=True)
        else:
            trueLocation.sort(key = len, reverse=True)

        if len(trueLocation) == 0:
            print(tagged[i])
            print(res_lower)
        # for ekonomi
        trueNegara = []
        for j in range(len(propn_list)):
            if propn_list[j].lower() in negara:
                trueNegara.append(propn_list[j].lower())
        temp_prov = '-'
        if trueNegara and n == 'ekonomi':
            if trueNegara[0] == 'indonesia':
                temp_prov = 'dki jakarta'
            countries.append(trueNegara[0])
        else:
            countries.append('-')

        if trueLocation:
            # print(i, trueLocation[0], '||', berita['ner_kec'], berita['ner_kab'], berita['ner_prov'])
            if len(trueLocation[0]) == 3:
                locations.append([trueLocation[0][0], trueLocation[0][1], trueLocation[0][2]])
            elif len(trueLocation[0]) == 2:
                locations.append(['-',trueLocation[0][0], trueLocation[0][1]])
            elif len(trueLocation[0]) == 1:
                locations.append(['-', '-', trueLocation[0][0]])
        else:
            # print(i, '-'*10, berita['ner_kec'], berita['ner_kab'], berita['ner_prov'])
            locations.append(['-', '-', temp_prov])
            
    # negara
    # if n == 'ekonomi':

    locations1 = []
    locations2 = []
    locations3 = []
    for i in range(len(locations)):
        locations1.append(locations[i][0])
        locations2.append(locations[i][1])
        locations3.append(locations[i][2])

    # ----------------------------------- WHO

    entities = []

    for i in range(len(tagged)):
        # print(tagged[i])
        propn_list = []
        idx_list = []
        
        # save each propn words to list
        for j in range(1, len(tagged[i])):
            teks = tagged[i].tokens[j].text.lower()
            if tagged[i].tokens[j].tag == 'PROPN' and tagged[i].tokens[j].text[0].isupper() and punct_check(teks) and \
                teks not in day and teks not in month and teks not in kata_excl and teks not in kec and teks not in kab and toOriginal(teks) not in prov:
                idx_list.append(j)
                propn_list.append(tagged[i].tokens[j].text)
        # print(propn_list)
        right_idx = []
        left_idx = []
        all_propn = []

        all_propn, left_idx, right_idx = ngrams_maker(propn_list, idx_list)
        # print(all_propn, left_idx, right_idx)
        
        filtered_propn = []
        for j in range(len(all_propn)):
            lower_propn = all_propn[j].lower()
            if lower_propn in kec and lower_propn in kab and lower_propn in prov:
                continue
            # AUX = adalah, telah
            # PART = tidak
            # ADP = di, ke, dalam
            # DET = ini, itu, tersebut
            # SCONJ = saat

            # check left
            if left_idx[j] - 2 >= 0:
                left_tag = tagged[i].tokens[left_idx[j] - 1].tag
                left_text = tagged[i].tokens[left_idx[j] - 1].text
                left_left_tag = tagged[i].tokens[left_idx[j] - 2].tag
                if left_tag == 'PROPN':
                    temp = find_left_tag_text(tagged[i], left_idx[j] - 1)
                    if temp:
                        left_tag = temp[0]
                        left_text = temp[1]
                        temp = find_left_tag_text(tagged[i], temp[2] - 1)
                        left_left_tag = temp[0]
                        left_left_text = temp[1]
                if left_tag == 'VERB' or (left_left_tag == 'VERB' and left_tag == 'AUX'):
                    if (right_idx[j] + 1 < len(tagged[i]) and (tagged[i].tokens[right_idx[j] + 1].tag == 'ADP' or tagged[i].tokens[right_idx[j] + 1].tag == 'NUM')):
                        continue
                    filtered_propn.append(all_propn[j])
                    continue
            # check right
            if right_idx[j] + 2 < len(tagged[i]):
                right_tag = tagged[i].tokens[right_idx[j] + 1].tag
                right_right_tag = tagged[i].tokens[right_idx[j] + 2].tag
                right_text = tagged[i].tokens[right_idx[j] + 1].text
                right_right_text = tagged[i].tokens[right_idx[j] + 2].text
                if right_tag == 'PROPN':
                    temp = find_right_tag_text(tagged[i], right_idx[j] + 1)
                    if temp:
                        right_tag = temp[0]
                        right_text = temp[1]
                        temp = find_right_tag_text(tagged[i], temp[2] + 1)
                        right_right_tag = temp[0]
                        right_right_text = temp[1]
                if right_tag == 'VERB' or \
                    (right_tag == 'PUNCT' and right_text == '.') or \
                    ((right_tag == 'AUX' or right_tag == 'PART' or right_tag == 'SCONJ') and right_right_tag == 'VERB'):
                    if (left_idx[j] - 1 >= 0 and tagged[i].tokens[left_idx[j] - 1].tag == 'NUM') or \
                        tagged[i].tokens[right_idx[j] + 1].text == 'ada':
                        continue
                    filtered_propn.append(all_propn[j])
                    continue
                # check both
                if left_idx[j] - 2 >= 0:
                    left_tag = tagged[i].tokens[left_idx[j] - 1].tag
                    left_text = tagged[i].tokens[left_idx[j] - 1].text
                    left_left_tag = tagged[i].tokens[left_idx[j] - 2].tag
                    right_tag = tagged[i].tokens[right_idx[j] + 1].tag
                    right_text = tagged[i].tokens[right_idx[j] + 1].text
                    right_right_tag = tagged[i].tokens[right_idx[j] + 2].tag
                    if right_tag == 'PROPN':
                        temp = find_right_tag_text(tagged[i], right_idx[j] + 1)
                        if temp:
                            right_tag = temp[0]
                            right_text = temp[1]
                            temp = find_right_tag_text(tagged[i], temp[2] + 1)
                            right_right_tag = temp[0]
                            right_right_text = temp[1]
                    if left_tag == 'PROPN':
                        temp = find_left_tag_text(tagged[i], left_idx[j] - 1)
                        if temp:
                            left_tag = temp[0]
                            left_text = temp[1]
                            temp = find_left_tag_text(tagged[i], temp[2] - 1)
                            left_left_tag = temp[0]
                            left_left_text = temp[1]
                    #add sconj to right tag
                    if left_tag == 'NOUN' and ((left_text == 'kata' and right_tag == 'PUNCT') or right_tag == 'DET' or (right_tag == 'ADP' and right_text == 'dalam' or right_text == 'di' or right_text == 'kepada') or right_text == ',') or \
                        (left_tag == 'PUNCT' and left_text == ',' and ((right_tag == 'PUNCT' and right_text == ',') or right_tag == 'ADP')) or \
                        (left_tag == 'PUNCT' and left_text == '.' and right_tag == 'PUNCT' and right_text == ','):
                        filtered_propn.append(all_propn[j])
            
        filtered_propn = dedup_list(filtered_propn)
        # print(filtered_propn)
        entities.append(", ".join(filtered_propn))
    
    tp = [0]*len(tagged)


    dict = {'who_n':entities,
            'tp_who_n':tp,
            'tn_who_n':tp,
            'who_o':berita['ner_who'].iloc[0:len(tagged)],
            'tp_who_o':tp,
            'tn_who_o':tp,
            'when_n':dates,
            'tp_when_n':tp,
            'tn_when_n':tp,
            'when_o':berita['ner_when'].iloc[0:len(tagged)],
            'tp_when_o':tp,
            'tn_when_o':tp,
            'new_kec':locations1,
            'new_kab':locations2,
            'new_prov':locations3,
            'tp_where_n':tp,
            'tn_where_n':tp,
            'old_kec':berita['ner_kec'].iloc[0:len(tagged)],
            'old_kab':berita['ner_kab'].iloc[0:len(tagged)],
            'old_prov':berita['ner_prov'].iloc[0:len(tagged)],
            'tp_where_o':tp,
            'tn_where_o':tp,
            'desc':berita['berita_desc'].iloc[0:len(tagged)]
            }
    if n == 'ekonomi':
        dict = {'who_n':entities,
            'tp_who_n':tp,
            'tn_who_n':tp,
            'who_o':berita['ner_who'].iloc[0:len(tagged)],
            'tp_who_o':tp,
            'tn_who_o':tp,
            'when_n':dates,
            'tp_when_n':tp,
            'tn_when_n':tp,
            'when_o':berita['ner_when'].iloc[0:len(tagged)],
            'tp_when_o':tp,
            'tn_when_o':tp,
            'new_kec':locations1,
            'new_kab':locations2,
            'new_prov':locations3,
            'new_neg':countries,
            'tp_where_n':tp,
            'tn_where_n':tp,
            'old_kec':berita['ner_kec'].iloc[0:len(tagged)],
            'old_kab':berita['ner_kab'].iloc[0:len(tagged)],
            'old_prov':berita['ner_prov'].iloc[0:len(tagged)],
            'old_neg':berita['ner_neg'].iloc[0:len(tagged)],
            'tp_where_o':tp,
            'tn_where_o':tp,
            'desc':berita['berita_desc'].iloc[0:len(tagged)]
            }

    df_final = pd.DataFrame(dict)
    # print(df_loc)
    # df_final.to_csv(PATH + 'whowhenwhere_test_result_' + n + '.csv')
    df_final.to_excel(PATH + 'whowhenwhere_test_result_' + n + '_v2.xlsx', index=False)

230 <class 'str'>
2022 <class 'str'>
2022 <class 'str'>
202 <class 'str'>
1964 <class 'str'>
1950 <class 'str'>
1950 <class 'str'>
1964 <class 'str'>
1959 <class 'str'>
20229 <class 'str'>
2022 <class 'str'>
6391999026 <class 'str'>
Sentence[266]: "\-  Bahu Jembatan Pamuruyan, penghubung Sukabumi-Bogor ambles. Bunyi beton berderak saat ambles mengagetkan warga yang berada di sekitar lokasi.  Diketahui jembatan itu berusia puluhan tahun. Saat ini pihak Kementerian PUPR melakukan pembangunan duplikasi jembatan tersebut. Lokasi jembatan Pamuruyan 2 ini berdekatan dengan jembatan lama.  Warga mengaku sempat melihat truk besar yang melintas sebelum peristiwa itu terjadi. Namun jembatan itu memang memiliki intensitas cukup tinggi dan kerap dilalui berbagai jenis kendaraan, termasuk truk berukuran besar.     Dari pihak pelaksana pekerjaan Jembatan Pamuruyan 2 mengatakan, amblesnya jalan diduga karena keadaan kahar atau force majeure, bukan dipicu kesalahan tekhnis.  "Karena force majeure, kar

In [47]:
df_final

,who_n,tp_who_n,tn_who_n,who_o,tp_who_o,tn_who_o,when_n,tp_when_n,tn_when_n,when_o,...,new_kab,new_prov,tp_where_n,tn_where_n,old_kec,old_kab,old_prov,tp_where_o,tn_where_o,desc
11446,"Villa Robin Olsen, Hojbjerg, Tyrone Mings, Tot...",0,0,"ivan, harry, john mcginn",0,0,01/01/2023,0,0,01/01/2023,...,-,-,0,0,-,-,-,0,0,\-\r\n\r\nLaga Tottenham Hotspur vs Aston Vill...
8767,"Wibowo, Iptu Umbaran, Dilansir, Wakhid, Kartik...",0,0,"wakhid, wibowo",0,0,17/12/2022,0,0,17/12/2022,...,lamongan,jawa timur,0,0,-,lamongan,jawa timur,0,0,\-\r\n\r\nIptu Umbaran menjadi sorotan publik ...
8825,"Assegaf Razak, Kenzo, PSM, PSM Bernardo Tavare...",0,0,-,0,0,19/12/2022,0,0,17/12/2022,...,bantul,di yogyakarta,0,0,-,makassar,sulawesi selatan,0,0,\-\r\n\r\nDuel gelandang asal Jepang bakal ter...
8857,"UEFA Europa League, Grup F, Maroko Vs Belgia, ...",0,0,"penampilannya, ziyech",0,0,27/11/2022,0,0,27/11/2022,...,-,-,0,0,-,-,-,0,0,\-\r\n\r\nHakim Ziyech bersinar bersama timnas...
11435,"Liga Inggris, WIB, VAR, Chamberlain, Mohamed S...",0,0,harvey elliott,0,0,03/01/2023,0,0,03/01/2023,...,-,-,0,0,-,-,-,0,0,\-\r\n\r\nLiverpool membuka tahun 2023 dengan ...
8824,"Assegaf Razak, PSS, PSM, Bernardo Tavares, Ram...",0,0,-,0,0,19/12/2022,0,0,15/12/2022,...,bantul,di yogyakarta,0,0,-,makassar,sulawesi selatan,0,0,\-\r\n\r\nNasib PSM Makassar di persaingan pap...
11448,"Trisakti Sadarehe, Kompol Edy Kusmawan, Bojong...",0,0,"edy, ari, linggasana, agus, ibrahim tompo, adi...",0,0,30/12/2022,0,0,23/12/2022,...,sukabumi,jawa barat,0,0,-,-,jawa barat,0,0,\-\r\n\r\nBeragam peristiwa terjadi di Jawa Ba...
8833,"Madura United, PSM, Tony Ho, Tony, Pemain Juku...",0,0,-,0,0,19/12/2022,0,0,17/12/2022,...,bantul,di yogyakarta,0,0,-,semarang,jawa tengah,0,0,\-\r\n\r\nPSM Makassar akan menjamu PSIS Semar...
8694,"Assegaf Razak, Tavares, United, PSM, Tim Raman...",0,0,agung mannan,0,0,23/12/2022,0,0,23/12/2022,...,bantul,di yogyakarta,0,0,-,bantul,di yogyakarta,0,0,\-\r\n\r\nPSM Makassar berhasil meraup satu po...
8982,"Grup F, Timothy Castagne, WIB, Roman Saiss, VA...",0,0,"hazard, achraf hakimi, sofyan, boufal",0,0,27/11/2022,0,0,27/11/2022,...,-,-,0,0,-,-,-,0,0,\-\r\n\r\nmemetik kemenangan atas di laga kedu...
